In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix, confusion_matrix
from sklearn.ensemble import BaggingClassifier
import matplotlib.pyplot as plt
import pandas as pd

def rmMissing(df: pd.DataFrame, header: list):
    idx = []
    for i in range(1,len(df.columns)):
        if (df[df.columns[i]].dtypes == "float64") or (sum(df[df.columns[i]]>1000)):
            if len(df.loc[df[df.columns[i]]==0]):
                idx.append(i)
    for col in df[df.columns[idx]].columns:
        del df[col]
        headers.remove(col)

df = pd.read_csv("dat.csv")
del df["chip"]
headers = ["cad", "alcohol_intake", "fish_oil_intake", "close_to_road", "chest_pain", "high_bp", "high_chol", "diabetes", "location", "bmi", "age", "gender", "pca1", "pca2", "pca3", "walking_freq", "time_tv", "time_sleep", "var_sleep", "townsend", "veggie_intake", "bread_intake", "tea_intake", "dia_pressure", "sys_pressure", "pulse", "num_veh", "inv_maj_road_dist", "inv_road_dist", "no2_2005", "no2_2006", "no2_2007", "no2_2010", "no_2010", "pm10_2007", "len_maj_road", "traffic_load_man","traffic_int_maj", "traffic_int_road","day_noise", "night_noise"]
df.columns = headers
rmMissing(df, headers)
x = df.iloc[:,1:]

In [ ]:
"""Full tree with pruning"""
mode = "Entropy"
name_modifier = "Balanced"#", Full Set of Covariates" #
bal = "balanced" # None
X_train, X_test, Y_train, Y_test = train_test_split(x, df.iloc[:,0], random_state=2)
entropy_tree = DecisionTreeClassifier(criterion=mode.lower(), class_weight=bal)
et_path = entropy_tree.cost_complexity_pruning_path(X_train, Y_train)
ccp_alphas, impurities = et_path.ccp_alphas, et_path.impurities

fig, ax = plt.subplots(3,1, figsize=(7,8))
ax[0].plot(ccp_alphas[:], impurities[:], marker='o', drawstyle="steps-post", markerfacecolor="r", color="k")
ax[0].set_xlabel(r"$\alpha$")
ax[0].set_ylabel("Total Impurity of Leaves")
ax[0].set_title("Total Impurity vs "+ r'$\alpha$' +", " + mode + name_modifier)

clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha, criterion=mode.lower(), class_weight=bal)
    clf.fit(X_train, Y_train)
    clfs.append(clf)
node_counts = [clf.tree_.node_count for clf in clfs]
depth = [clf.tree_.max_depth for clf in clfs]

ax[1].plot(ccp_alphas, depth, marker='o', drawstyle="steps-post", markerfacecolor="r",color="k")
ax[1].set_xlabel(r'$\alpha$')
ax[1].set_ylabel("Depth of Tree")
ax[1].set_title("Depth vs " + r'$\alpha$' + ", " + mode + name_modifier)

train_scores = [clf.score(X_train, Y_train) for clf in clfs]
test_scores = [clf.score(X_test, Y_test) for clf in clfs]

ax[2].set_xlabel(r'$\alpha$')
ax[2].set_ylabel("Accuracy")
ax[2].set_title("Accuracy vs " + r'$\alpha$' +", " + mode + name_modifier)
ax[2].plot(ccp_alphas, train_scores, marker='o', label="Train",
        drawstyle="steps-post")
ax[2].plot(ccp_alphas, test_scores, marker='o', label="Test",
        drawstyle="steps-post")
ax[2].legend()
fig.tight_layout()
plt.show()

In [ ]:
cm = []
weighted_bias = []
true_pos_corr = []
fig, ax = plt.subplots()

for i in range(len(ccp_alphas)):
    cm.append(confusion_matrix(clfs[i].predict(X_test), Y_test))
    weighted_bias.append(((cm[i][1][0] + cm[i][1][1]) - (cm[i][0][1] + cm[i][1][1]))/((cm[i][0][1] + cm[i][1][1])))
    true_pos_corr.append(cm[i][1][1] / (cm[i][0][1] + cm[i][1][1]))
    plt.text(true_pos_corr[i],weighted_bias[i], "%.4G" % ccp_alphas[i])


ax.plot(true_pos_corr,weighted_bias,  marker='o', drawstyle="steps-post", markerfacecolor="r",color="k")
ax.set_xlabel('True Positive Accuracy')
ax.set_ylabel("Bias")
ax.set_title("Bias vs True Positive Accuracy, " + mode + name_modifier)
plt.show()


In [ ]:
disp = plot_confusion_matrix(clfs[11],X_test, Y_test)
disp.ax_.set_title("Confusion Matrix, "+mode+", " + r'$\alpha$' + '=' + str(ccp_alphas[11]))

In [ ]:
from pydotplus.graphviz import graph_from_dot_data
from sklearn.tree import export_graphviz

dot_data = export_graphviz(                           # Create dot data
    tree, filled=True, rounded=True,
    class_names=['nCAD','CAD'],
    feature_names=headers[1:],
    out_file=None
)

graph = graph_from_dot_data(dot_data) 
graph.write_png('tree.png')                           # Write graph to PNG image

In [ ]:
l = [k/100 for k in range(0,24, 1)]
for i in range(1,51,2):
    bcm = []
    bweighted_bias = []
    btrue_pos_corr = []
    fig, ax = plt.subplots()
    print(i)
    for j in l:
        bt = DecisionTreeClassifier(random_state=0, max_features=None, ccp_alpha=j, criterion=mode.lower(), class_weight=bal)
        bag = BaggingClassifier(base_estimator=bt, n_estimators=i, max_samples=1.0)
        bag = bag.fit(x, df.iloc[:,0])
        bcm = confusion_matrix(bag.predict(x),  df.iloc[:,0] )
        bweighted_bias = (((bcm[1][0] + bcm[1][1]) - (bcm[0][1] + bcm[1][1]))/((bcm[0][1] + bcm[1][1])))
        btrue_pos_corr = (bcm[1][1] / (bcm[0][1] + bcm[1][1]))
        plt.text(btrue_pos_corr,bweighted_bias, "%.3G" % j)

    ax.plot(btrue_pos_corr,bweighted_bias,  marker='o', drawstyle="steps-post", markerfacecolor="r",color="k")
    ax.set_xlabel('True Positive Accuracy')
    ax.set_ylabel("Bias")
    ax.set_title("Bias vs True Positive Accuracy, " + mode + ', Num Estimators=' + str(i))
    plt.show()

In [ ]:
confusion_matrix(bag.predict(x),  df.iloc[:,0] )
